# Prospective evaluation of policies to reduce energy demand reduction policies for residential space heating

**Objective**:
The present exercise values, with the help of the technical-economic model Res-IRF, the effect of the main energy efficiency subsidies (tax credit for energy transition, co-payment of zero-rate, VAT at reduced rate, energy saving certificates), the carbon tax and the thermal regulation on the energy demand for heating homes in France.

About **this Notebook**:  
This notebook is an example of post-treatment to **assess public policies**.  
It takes as input a folder with at least 2 output of Res-IRF run. Each represents a scenario: one considering the public policies, and one not.

**Step 1: Run Res-IRF with Multiple scenarios.**   
**Step 2: Launch this Notebook and put folder name that is a Res-IRF ouput folder.**


About **Res-IRF**:  
The Res-IRF model is a tool for simulating energy consumption for space heating in the French residential sector.  
Its main characteristic is to integrate a detailed description of the energy performance of the dwelling stock with a rich description of household behaviour.

## Methodology for evaluating public policies

The simulations are based on the following principles:
1. All instruments apply from 2012 (except the carbon tax which applies from 2014) and are maintained until 2050 ; they are therefore taken into account in the calibration of the model;
2. The different instruments are fully cumulative with each other;
3. The instruments operate at full capacity; in particular, a household who achieves an innovation takes all the aids for which he or she is eligible;
4. The instruments apply equally to private and social housing;
5. Incentives are perfectly transmitted to households, without being captured by energy efficiency sellers in the form of an increase in the base price; this mechanism is based on an assumption of perfect competition in the energy innovation sector


More generally, the study aims at least as much to illustrate qualitatively the mechanisms by which the different instruments operate as to provide a precise quantitative assessment of their impact.

### Evaluation indicator

Version 3.0 of Res-IRF allows for a multi-criteria evaluation of policies. 
- The **effectiveness** of an intervention - instrument or mix of instruments - is measured as its ability to achieve the **objectives** assigned to it - in this case, the five objectives defined bellow. 
- To this indicator, we now add the **efficiency** of an intervention, valued as its capacity to achieve a certain **objective at the lowest cost**. This notion only applies to incentive instruments and therefore not to thermal regulation. Two efficiency indicators are calculated: 
    - **cost-efficiency** and,
    - **leverage effect**. 
- Finally, the differentiation of households by income category allows us to evaluate the **distributional effects**.

**Objectives**:

1. Reduction of **final energy consumption by 20% by 2030** and **50% by 2050** compared to 2012 - this objective is applied here to residential heating consumption;
2. Energy renovation of **500,000 homes per year**, including 120,000 in social housing;
3. Disappearance of **G and F** energy performance housing **by 2025**;
4. Renovation of the **entire housing stock** to the "low-energy building" level or similar **by 2050**- a requirement interpreted here as a minimum performance equal to the B label of the DPE;
5. Reducing **fuel poverty by 15% by 2020**.


#### Effectivness
Evaluating the additional effect of an intervention requires the formulation of a counterfactual scenario, without the intervention under consideration.

Two counterfactual situations, obtained by two different methods, are of particular interest because they allow us to limit the impact of each instrument:
- Method TP-1: the TP scenario is compared to an alternative scenario without the instrument under consideration. The difference between the two scenarios gives the impact of the instrument in interaction with all the other instruments.
- ZP+1 method: the ZP scenario is compared to an alternative scenario with the instrument under consideration. The difference between the two scenarios gives the pure impact of the instrument.

# Code

To assess the effectivity of a public policy, this Notebook need as input at least 2 scenarios (one with the public policy and another one withtout)

## Reading input

In [1]:
name_folder = '../output/scenarios_input_2012'

Enter common prefixe to policies scenario:  
*Example: enter 'CITE' if scenarios are called 'CITE-2025', 'CITE-2030', ...*

NB: Reference should be one of the scenario name. 

### Code

In [2]:
import os
import pickle
import matplotlib.pyplot as plt
# from itertools import product
import numpy as np
import seaborn as sns
import sys

sys.path.insert(0, "..")

from ui_utils import *

In [3]:
name_file = os.path.join('input', 'colors.json')
with open(name_file) as file:
    colors = json.load(file)

In [4]:
scenarios = [f for f in os.listdir(os.path.join(name_folder)) if os.path.isdir(os.path.join(name_folder, f))]
folders = {scenario: os.path.join(name_folder, scenario) for scenario in scenarios} 

In [5]:
sns.set_palette(sns.color_palette('husl', len(scenarios)))

In [6]:
summaries = {scenario: pd.read_csv(os.path.join(folders[scenario], 'summary.csv'), index_col=[0]) for scenario in scenarios}
summaries = reverse_nested_dict(summaries)
summaries = {key: pd.DataFrame(item) for key, item in summaries.items()}

In [7]:
output_stock = {scenario: pickle.load(open(os.path.join(folders[scenario], 'output_stock.pkl'), 'rb')) for scenario in scenarios}
output_stock = reverse_nested_dict(output_stock)

# Efficiency

To estimate the marginal effect of the instrument in year $t$, we compare scenarios with and without the instrument in year $t$.

## Cost effectiveness

The cost-efficiency indicator relates the costs of the incentive to its effectiveness measured in terms of energy savings. The cost considered corresponds to the tax expenditure in the case of subsidies and the tax revenue in the case of the tax (i.e., negative cost).

The indicator is calculated here in conventional and real energy.   
The conventional energy metric makes it possible to eliminate the heterogeneous behavioral effects between households. It leads to **overestimating the effectiveness of subsidies by ignoring the rebound effect** they generate and **underestimating the effectiveness of energy taxes**, which on the contrary induce a sobriety effect.

To calculate the cost-efficiency indicator $CE$, the energy savings $\Delta E_t$ between the scenarios are compared with the incentive present or absent in year $t$ (but in both cases present until year $t-1$), by applying a discount factor $FA$ :

$$CE_t = \frac{Incitation_t}{\Delta E_t . FA}$$

df = Standard - wo/ policies  
df['Consumption actual (kWh)'] < 0  
df['Subsidies (€)'] > 0  

### Energy consumption

In [24]:
df = summaries['Consumption actual (kWh)'].copy()
list_years = [int(re.search("20[0-9][0-9]", column)[0]) for column in df.columns if re.search("20[0-9][0-9]", column)]
marginal_consumption_actual = pd.Series([(df['Standard'] - df['{}-{}'.format(policy, year)]).loc[year] for year in list_years], index=list_years)

KeyError: 'Standard'

In [25]:
df = summaries['Consumption conventional (kWh)'].copy()
list_years = [int(re.search("20[0-9][0-9]", column)[0]) for column in df.columns if re.search("20[0-9][0-9]", column)]
marginal_consumption_conventional = pd.Series([(df['Standard'] - df['{}-{}'.format(policy, year)]).loc[year] for year in list_years], index=list_years)

KeyError: 'Standard'

### Emission

In [ ]:
df = summaries['Emission (gCO2)'].copy()
list_years = [int(re.search("20[0-9][0-9]", column)[0]) for column in df.columns if re.search("20[0-9][0-9]", column)]
marginal_emission = pd.Series([(df['Standard'] - df['{}-{}'.format(policy, year)]).loc[year] for year in list_years], index=list_years)

### Policy cost

In [ ]:
df = summaries['Reduced tax (€)'].copy()
df.fillna(0, inplace=True)
list_years = [int(re.search("20[0-9][0-9]", column)[0]) for column in df.columns if re.search("20[0-9][0-9]", column)]
marginal_subsidies = pd.Series([(df['Standard'] - df['{}-{}'.format(policy, year)]).loc[year] for year in list_years], index=list_years)

### Extension of energy and emission savings

Using a discount rate of 4% and a lifetime of 26 years, which corresponds to the average of the operations carried out within the framework of the CEE on the perimeter of the residential building, the factor $FA$ is
taken equal to 16.6.

In [ ]:
discount_rate = 0.04
lifetime = 26
discount_factor = discount_factor = (1 - (1 + discount_rate) ** -lifetime) / discount_rate
print('Discount factor: {:.1f}'.format(discount_factor))

### Results

In [ ]:
cost_effectivness = abs(pd.concat((marginal_subsidies / (marginal_consumption_actual * discount_factor), marginal_subsidies / (marginal_consumption_conventional * discount_factor)), axis=1))
cost_effectivness.columns = ['Actual', 'Conventional']
cost_effectivness.sort_index(inplace=True)
print(cost_effectivness)         

## Leverage effect

The leverage effect $EL$ relates the efficiency of the instrument, measured in terms of investment expenses, to the cost of the incentive. A leverage effect of 1 implies that one euro of public money (grant expenditure or tax revenue) induces an additional investment of one euro.


The formula used applies to all participants and relates the cost of the incentive to the surplus of investment $\Delta Inv$ induced by the policy, measured as the difference between two scenarios with the incentive absent or present in year $t$ (but in both cases present until year $t-1$):
$$EL_t = \frac{\Delta Inv_t}{Incitation_t}$$


In [ ]:
df = summaries['Capex renovation (€)'].copy()
df.fillna(0, inplace=True)
list_years = [int(re.search("20[0-9][0-9]", column)[0]) for column in df.columns if re.search("20[0-9][0-9]", column)]
marginal_investment = pd.Series([(df['Standard'] - df['{}-{}'.format(policy, year)]).loc[year] for year in list_years], index=list_years)

In [ ]:
leverage = marginal_investment / marginal_subsidies
leverage.sort_index(inplace=True)
print(leverage)

# Goals
Objectives are based on stock output (aggregated and segmented) and transition output.

1. Reduction of **final energy consumption by 20% by 2030** and **50% by 2050** compared to 2012 - this objective is applied here to residential heating consumption;
2. Energy renovation of **500,000 homes per year**, including 120,000 in social housing;
3. Disappearance of **G and F** energy performance housing **by 2025**;
4. Renovation of the **entire housing stock** to the "low-energy building" level or similar **by 2050**- a requirement interpreted here as a minimum performance equal to the B label of the DPE;
5. Reducing **fuel poverty by 15% by 2020**.

In [ ]:
policy = 'CITE'

Objective is to estimate the impact of policies on some variable like the energy consumption or C02 emissions.  
Method consists to compare a scenario with the policy to a scenario without the policy.  
However, some variables persist over time, like energy consumption. Investing in renovation will create energy savings for more than 30 years.  
As the model will not consider the end, variables values need to be extended.

## Energy saving

In [ ]:
df = pd.concat((summaries['Consumption actual (kWh)']['Standard'], summaries['Consumption actual (kWh)']['{}-1'.format(policy)]), axis=1) / 10**9

In [ ]:
simple_pd_plot(df, 'Years', 'Consumption actual (TWh)')

### Simple difference
Need to be extented after the end of last studied year, as investment duration is larger than the sudies duration.  
With this method, it's not possible to precisely estimate how long difference should be extended. 
If investment duration is 30 years.  
- renovations that occur first (in 2019) need to be extended by 10 years at the end of the study.   
- late renovations (in 2039) need to be extended by 30 years at the end of the study.  


In [ ]:
simple_diff = df['Standard'] - df['{}-1'.format(pre)]
simple_pd_plot(simple_diff, 'Years', 'Energy saving (TWh)')

### Double difference
Renovations differences between 2 scenarios are responsible for marginal energy consumption difference.
Renovations last n years from the time investment is made so we can estimate total energy saving.

However, as energy prices and households available income evolve, heating intensity change.  
Energy saving of one renovation will also evolve, yet we neglect this, as the degradation of equipment.


- The marginal effect of an increase in income on the heating intensity is independent from conventional energy consumption.

In [ ]:
double_diff = simple_diff.diff()
double_diff.iloc[0] = simple_dif.iloc[0]
simple_pd_plot(double_diff, 'Years', 'Marginal energy saving (TWh)')

In [ ]:
discount_rate = 0.04
lifetime = 26
discount_factor = discount_factor = (1 - (1 + discount_rate) ** -lifetime) / discount_rate
print('Discount factor: {:.1f}'.format(discount_factor))

In [ ]:
energy_saving = - (double_diff * discount_factor).sum()
print('Energy saving: {:.2f} TWh'.format(energy_saving))

## Buildings revovation

In [ ]:
df = pd.concat((summaries['Flow transition renovation']['Standard'], summaries['Flow transition renovation']['{}-1'.format(policy)]), axis=1) / 10**3



In [ ]:
simple_pd_plot(df, 'Years', 'Flow renovation (Thousands)')
simple_pd_plot(df.cumsum(), 'Years', 'Cumulated number of renovation (Thousands)')

### Policy marginal effect on buildings retrofit

In [ ]:
simple_diff = df['Standard'] - df['{}-1'.format(policy)]
simple_pd_plot(simple_diff, 'Years', 'Number of additional renovations (Thousands)')

In [ ]:
additional_renovation = simple_diff.sum()
mean_additional_renovation = simple_diff.mean()

print('Additional renovations: {:.0f}k'.format(additional_renovation))
print('Mean additional renovations by year: {:.0f}k/yr'.format(mean_additional_renovation))

## G and F buildings in 2025

In [ ]:
df = (detailed['Stock F (Thousands)'] + detailed['Stock G (Thousands)']).loc[:, ['Standard', '{}-1'.format(policy)]] / 10**3
simple_pd_plot(df, 'Years', 'F and G buildings (Millions)')


### Share of F and G buildings in the total buildings stock

In [ ]:
df = (df * 10**3 / detailed['Stock (Thousands)']).loc[:, ['Standard', '{}-1'.format(policy)]]
simple_pd_plot(df, 'Years', 'Share of F and G buildings')

### Policy marginal effect on F and G buildings

In [ ]:
simple_diff = df['Standard'] - df['{}-1'.format(policy)]
simple_pd_plot(simple_diff, 'Years', 'Marginal F and G buildings decrease (%)')

In [ ]:
share_high_energy_buildings = - simple_diff.iloc[-1]
print('Marginal decreased of F and G buildings share: {:.2%}'.format(share_high_energy_buildings))

## Entire housing stock to the "low-energy building" level or similar by 2050

In [ ]:
df = detailed['Stock G (Thousands)'] + detailed['Stock F (Thousands)'] + detailed['Stock E (Thousands)'] + detailed['Stock D (Thousands)'] + detailed['Stock C (Thousands)']
df = detailed['Stock (Thousands)'] - df
df = df.loc[:, ['Standard', '{}-1'.format(policy)]] / 10**3

simple_pd_plot(df, 'Years', 'Number of low energy buildings (Millions)')

### Share of 'low-energy-building' in the total buildings stock.

In [ ]:
df = (df * 10**3 / detailed['Stock (Thousands)'].loc[:, ['Standard', '{}-1'.format(policy)]])
simple_pd_plot(df, 'Years', 'Share of low energy buildings (%)')

### Policy marginal effect on 'low energy building'

In [ ]:
simple_diff = df['Standard'] - df['{}-1'.format(policy)]
simple_pd_plot(simple_diff, 'Years', 'Additional share of low energy buildings (%)')

In [ ]:
share_low_energy_buildings = simple_diff.iloc[-1]
print('Marginal increased of low energy buildings: {:.2%}'.format(share_low_energy_buildings))

## Reducing fuel poverty by 15% by 2020

In [ ]:
df = summaries['Energy poverty'].loc[:, ['Standard', '{}-1'.format(policy)]] / 10**6
simple_pd_plot(df, 'Years', 'Number of fuel poverty buildings (Millions)')

### Share of 'fuel poverty' buildings in the total buildings stock

In [ ]:
df = (df * 10**6 / summaries['Stock'].loc[:, ['Standard', '{}-1'.format(policy)]])
simple_pd_plot(df, 'Years', 'Share of low energy buildings (%)')

### Simple diff

In [ ]:
simple_diff = df['Standard'] - df['{}-1'.format(policy)]
simple_pd_plot(simple_diff, 'Years', 'Reducing number of fuel poverty buildings')

### Policy marginal effect on energy poverty

In [ ]:
share_energy_poverty = - simple_diff.iloc[-1]
print('Marginal decreased of energy poverty: {:.2%}'.format(share_energy_poverty))